<h1 align=center><font size = 5>Clustering Neighborhoods in Toronto</font></h1>

Firs we do all required imports and we install beautifulsoup4

In [1]:
import requests
import pandas as pd
!pip install beautifulsoup4
from bs4 import BeautifulSoup
import folium # map rendering library
import numpy as np
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

     |████████████████████████████████| 122kB 3.4MB/s eta 0:00:01


<br>
<br>
Now we perform the request to the Wikipedia website to get all html content

In [2]:
latitude = 43.6468467
longitude = -79.3820296
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641"
page = requests.get(url)
page

<Response [200]>

<br>
<br>
We have to find the content of the table we want to clean

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find_all('table')[0]
data = []

for child in soup.find_all('table')[0].children:
    for td in child:
        if isinstance(td, str) == False:
            data.append(td.text.split('\n'))        
for i in range(len(data)):
    while("" in data[i]) :
        data[i].remove("")

<br>
<br>
We now build our DataFrame with the list "data" that contains all the required information 

In [4]:
df = pd.DataFrame(data[1::]) #skipping first row to use as columns names
df.columns = data[0]
df.shape

(288, 3)

<br>
<br>
-Ignoring cells with a borough that is Not assigned.<br>
-If cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.<br>
-If more than one neighborhood exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [5]:
df = df.loc[df['Borough']!='Not assigned']
df['Neighbourhood'].replace('Not assigned',df[df['Neighbourhood']=='Not assigned']['Borough'].values[0],inplace=True)
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [6]:
df.shape

(103, 3)

In [7]:
dfgeo = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
dfgeo.rename(columns = {'Postal Code': 'Postcode', 'Latitude': 'Latitude', 'Longitude':'Longitude'}, inplace = True)


In [8]:
dfgeo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<br>
<br>
We will merge the tables so we can have the Latitude and longitude.

In [9]:
df_merge = pd.merge(df, dfgeo, on='Postcode')

df_merge.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<br>
<br>
Now we will work with Boroughs that contain Toronto

In [10]:
subdf = df_merge[df_merge['Borough'].str.contains('Toronto')]
subdf = subdf.rename(columns = {'Postcode': 'Postcode', 'Borough': 'Borough','Neighbourhood':'Neighborhood','Latitude':'Latitude','Longitude':'Longitude'})
subdf

,Postcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


<br>
<br>
Let's create our folium map to visualize our locations

In [11]:
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(subdf['Latitude'], subdf['Longitude'], subdf['Borough'], subdf['Neighborhood']):
    label = '{} | {}'.format(borough,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<br>
<br>
Now we will enter our creds to work with Foursquare API 

In [ ]:
CLIENT_ID = '#########' # your Foursquare ID
CLIENT_SECRET = '########' # your Foursquare Secret
ACCESS_TOKEN = '########' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

<br>
<br>
Defining the function to perform the query for all locations selected to be analysed 

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=subdf['Neighborhood'], latitudes=subdf['Latitude'], longitudes=subdf['Longitude'])
toronto_venues.head()

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [16]:
toronto_venues.shape

(1561, 7)

<br>
<br>
Now we will count the Categories on the neighborhoods, then we will transform categorical variables into dummy/indicator variables.

In [17]:
toronto_venues.groupby('Neighborhood').count().sort_values(by=['Venue Category'], ascending=False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Harbourfront East,Toronto Islands,Union Station",100,100,100,100,100,100
"Commerce Court,Victoria Hotel",100,100,100,100,100,100
"Ryerson,Garden District",100,100,100,100,100,100
"First Canadian Place,Underground city",100,100,100,100,100,100
"Design Exchange,Toronto Dominion Centre",100,100,100,100,100,100
Stn A PO Boxes 25 The Esplanade,99,99,99,99,99,99
"Adelaide,King,Richmond",93,93,93,93,93,93
St. James Town,82,82,82,82,82,82
Church and Wellesley,71,71,71,71,71,71


In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

In [19]:
# move neighborhood column to the first column
toronto_onehot = toronto_onehot.reindex(columns=(['Neighborhood'] + list([a for a in toronto_onehot.columns if a != 'Neighborhood']) ))
toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<br><br>Now we'll set the dataframe information that will be used in our K-means model

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.010753,0.0,0.0,0.010753,...,0.0,0.0,0.0,0.010753,0.0,0.0,0.0,0.0,0.0,0.00
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.016667,...,0.0,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.0,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.04
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00


<br><BR>
Let's now get the 5 most common venues for the Neighborhoods we are analyzing

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Restaurant,Gym,Deli / Bodega
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Cheese Shop
2,"Brockton,Exhibition Place,Parkdale Village",Café,Bakery,Breakfast Spot,Performing Arts Venue,Coffee Shop
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Pizza Place,Auto Workshop,Recording Studio,Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Airport,Boat or Ferry


<br><br>
Time to create the Clusters using the Neighborhood and categories' mean table

In [23]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,0,"Adelaide,King,Richmond",Coffee Shop,Café,Restaurant,Gym,Deli / Bodega
1,0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Cheese Shop
2,0,"Brockton,Exhibition Place,Parkdale Village",Café,Bakery,Breakfast Spot,Performing Arts Venue,Coffee Shop
3,0,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Pizza Place,Auto Workshop,Recording Studio,Restaurant
4,0,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Airport,Boat or Ferry


In [25]:
toronto_merged = pd.merge(subdf, neighborhoods_venues_sorted, on='Neighborhood')
toronto_merged.to_csv("output.csv")

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [27]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,0,Health Food Store,Trail,Pub,Yoga Studio,Discount Store
1,East Toronto,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Restaurant
2,East Toronto,0,Fast Food Restaurant,Park,Gym,Sushi Restaurant,Ice Cream Shop
3,East Toronto,0,Coffee Shop,Café,Bakery,Gastropub,American Restaurant
5,Central Toronto,0,Gym,Breakfast Spot,Food & Drink Shop,Department Store,Hotel
6,Central Toronto,0,Coffee Shop,Clothing Store,Yoga Studio,Chinese Restaurant,Spa
7,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Gym,Sushi Restaurant
9,Central Toronto,0,Coffee Shop,Bagel Shop,Liquor Store,Restaurant,Bank
11,Downtown Toronto,0,Café,Coffee Shop,Park,Pizza Place,Restaurant
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Hotel


From previous analysis we understand that with only 5 clusters most of the boroughs fall in one cluster as there is a significant difference with The Beaches, Rosedale, Forest Hill North,Forest Hill West, Lawrence Park, Roselawn and Moore Park,Summerhill East